In [110]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

import warnings
import requests
import pickle

from timeit import default_timer as timer
from datetime import datetime

from pathlib import Path
from typing import Tuple, Callable, List

import imblearn
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTENC, SMOTE

from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay, precision_recall_curve
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance

import xgboost
from xgboost import XGBClassifier

In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from pathlib import Path

import warnings

warnings.filterwarnings('ignore')

In [112]:
# Obtiene el path actual
PROJECT_PATH = os.getcwd()

src = Path('src')
DATA_PATH = PROJECT_PATH / src

img = Path('img')
IMG_PATH = PROJECT_PATH / img

models = Path('models')
MODELS_PATH = PROJECT_PATH / models

reports = Path('reports')
REPORTS_PATH = PROJECT_PATH / reports

dirs = [DATA_PATH, IMG_PATH, MODELS_PATH, REPORTS_PATH]


for directory in dirs:
    if not directory.exists():
        directory.mkdir(parents=True)
        print(f'Directorio creado:\n{directory}\n')
    else:
        print(f'El directorio ya existe:\n{directory}\n')

El directorio ya existe:
C:\Users\Ignacio\JupyterScripts\Fintech_NC\src

El directorio ya existe:
C:\Users\Ignacio\JupyterScripts\Fintech_NC\img

El directorio ya existe:
C:\Users\Ignacio\JupyterScripts\Fintech_NC\models

El directorio ya existe:
C:\Users\Ignacio\JupyterScripts\Fintech_NC\reports



In [113]:
df_clients = pd.read_parquet(DATA_PATH/'clients.parquet')
df_tx = pd.read_parquet(DATA_PATH/'transactions.parquet')
df_sessions = pd.read_parquet(DATA_PATH/'app_sessions.parquet')
df_cl_app_features = pd.read_parquet(DATA_PATH/'client_app_features.parquet')

In [114]:
df_clients.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,2025-02-26,Clientes potenciales
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,NaT,Standard
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,2024-07-23,VIP
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,NaT,Nuevos - Poco Valor
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,NaT,Standard


In [115]:
df_clients = df_clients.drop(['RowNumber', 'Surname', 'Geography'], axis=1)
df_clients.head()

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label
0,15634602,619,Female,42,2,0.00,1,1,1,101348.88,1,2025-02-26,Clientes potenciales
1,15647311,608,Female,41,1,83807.86,1,0,1,112542.58,0,NaT,Standard
2,15619304,502,Female,42,8,159660.80,3,1,0,113931.57,1,2024-07-23,VIP
3,15701354,699,Female,39,1,0.00,2,0,0,93826.63,0,NaT,Nuevos - Poco Valor
4,15737888,850,Female,43,2,125510.82,1,1,1,79084.10,0,NaT,Standard


In [116]:
df_tx

,date,CustomerId,type,amount,amount_label,type_freq,frequency_label,amount_score,freq_score,tran_score,tran_score_cont,tran_label,max_tx_cap,amount_clipped,amount_ratio,amount_scaled
0,2023-09-19,15643158,CASH_OUT,134289.36,Medium,0.351663,High,2,3,5,2.300145,High,124481.822989,134289.36,0.082846,10312.80
1,2023-09-19,15785078,CASH_OUT,3716.84,Low,0.351663,High,1,3,4,1.600004,Low,114582.264492,3716.84,0.002022,231.74
2,2023-09-19,15700383,PAYMENT,23338.41,High,0.338146,Medium,3,2,5,2.700025,High,130630.168250,23338.41,0.014168,1850.78
3,2023-09-19,15761286,CASH_OUT,89888.35,Medium,0.351663,High,2,3,5,2.300097,Medium,137982.267193,89888.35,0.055362,7638.97
4,2023-09-19,15680855,CASH_IN,124555.78,Medium,0.219923,Medium,2,2,4,2.000135,Medium,131240.845531,124555.78,0.076821,10082.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6352030,2025-09-30,15696859,CASH_IN,82119.38,Medium,0.219923,Medium,2,2,4,2.000089,Medium,137827.490205,82119.38,0.050553,6967.60
6352031,2025-09-30,15793671,PAYMENT,17376.74,Medium,0.338146,Medium,2,2,4,2.000019,Medium,136943.350409,17376.74,0.010478,1434.87
6352032,2025-09-30,15661945,CASH_OUT,154712.79,Medium,0.351663,High,2,3,5,2.300167,High,113325.564397,154712.79,0.095488,10821.20
6352033,2025-09-30,15660735,CASH_IN,199797.12,Medium,0.219923,Medium,2,2,4,2.000216,Medium,139128.877629,199797.12,0.123395,17167.75


In [117]:
df_tx.columns

Index(['date', 'CustomerId', 'type', 'amount', 'amount_label', 'type_freq',
       'frequency_label', 'amount_score', 'freq_score', 'tran_score',
       'tran_score_cont', 'tran_label', 'max_tx_cap', 'amount_clipped',
       'amount_ratio', 'amount_scaled'],
      dtype='object')

In [118]:
df_tx = df_tx.drop(['amount', 'amount_label', 'type_freq',
       'frequency_label', 'amount_score', 'freq_score', 'tran_score',
       'tran_score_cont', 'tran_label', 'max_tx_cap', 'amount_clipped',
       'amount_ratio'], axis=1)

In [119]:
df_tx

,date,CustomerId,type,amount_scaled
0,2023-09-19,15643158,CASH_OUT,10312.80
1,2023-09-19,15785078,CASH_OUT,231.74
2,2023-09-19,15700383,PAYMENT,1850.78
3,2023-09-19,15761286,CASH_OUT,7638.97
4,2023-09-19,15680855,CASH_IN,10082.03
...,...,...,...,...
6352030,2025-09-30,15696859,CASH_IN,6967.60
6352031,2025-09-30,15793671,PAYMENT,1434.87
6352032,2025-09-30,15661945,CASH_OUT,10821.20
6352033,2025-09-30,15660735,CASH_IN,17167.75


Basado en tus propios gráficos, aquí hay características que seguramente serán muy predictivas:

* Características de Frecuencia y Recencia:

`dias_desde_ultima_transaccion`: Suele ser el predictor #1 de churn.

`total_transacciones_ultimos_30_dias`: ¿La actividad es reciente?

`total_transacciones_ultimos_90_dias`

`promedio_transacciones_por_semana`

* **Características de Monto (usando amount_scaled):**

`monto_total_transaccionado`

`monto_promedio_por_transaccion`

`monto_promedio_cash_out` (tu boxplot sugiere que esto es clave).

`monto_total_ultimos_30_dias`.

`desviacion_estandar_montos`: ¿Sus transacciones son consistentes o erráticas?

* Características de Comportamiento:

`ratio_cash_out_vs_total_tx`: ¿Qué proporción de sus operaciones son retiros?

`ratio_transfer_vs_total_tx`

`tipos_de_transaccion_unicos`: ¿Es un usuario que explora diferentes servicios?

In [120]:
df_tx

,date,CustomerId,type,amount_scaled
0,2023-09-19,15643158,CASH_OUT,10312.80
1,2023-09-19,15785078,CASH_OUT,231.74
2,2023-09-19,15700383,PAYMENT,1850.78
3,2023-09-19,15761286,CASH_OUT,7638.97
4,2023-09-19,15680855,CASH_IN,10082.03
...,...,...,...,...
6352030,2025-09-30,15696859,CASH_IN,6967.60
6352031,2025-09-30,15793671,PAYMENT,1434.87
6352032,2025-09-30,15661945,CASH_OUT,10821.20
6352033,2025-09-30,15660735,CASH_IN,17167.75


In [121]:
#1. Calcular la fecha de la última transacción por cliente en df_tx
last_tx_date = df_tx.groupby('CustomerId')['date'].max().reset_index(name='LastTransactionDate')

# 2. Filtrar clientes que abandonaron y unirlos con su última transacción
df_churned = df_clients[df_clients['Exited'] == 1].copy()
df_churned_check = df_churned.merge(last_tx_date, on='CustomerId', how='left')

# 3. Realizar la comprobación de integridad
# El error ocurre si la ExitDate es ANTES de la LastTransactionDate
df_churned_check['TemporalIntegrityError'] = df_churned_check['ExitDate'] < df_churned_check['LastTransactionDate']

# 4. Reportar los casos problemáticos (donde el error es True)
integrity_violations = df_churned_check[df_churned_check['TemporalIntegrityError'] == True]

print(f"Clientes que abandonaron ('Exited'=1): {df_churned.shape[0]}")
print(f"Clientes con violación de integridad temporal: {integrity_violations.shape[0]}")

if integrity_violations.empty:
    print("\n✅ ¡Validación completada! No se encontraron transacciones después de las fechas de salida registradas.")
else:
    print("\n🚨 ¡ATENCIÓN! Se encontraron violaciones de integridad. Primeros 5 casos:")
    print(integrity_violations[['CustomerId', 'ExitDate', 'LastTransactionDate']].head())
    print("Estas filas indican que el cliente tuvo una transacción posterior a su fecha de salida registrada ('ExitDate').")

Clientes que abandonaron ('Exited'=1): 1943
Clientes con violación de integridad temporal: 0

✅ ¡Validación completada! No se encontraron transacciones después de las fechas de salida registradas.


In [126]:
# 1. Definir la fecha máxima de actividad del dataset (simulated_today_date)
# Asumimos que tu DataFrame de actividad se llama df_tx (o df_sessions, o df_assigned)
max_date = df_tx['date'].max()

# 2. Inicializar la columna 'ObservationDate' en df_clients
df_clients['ObservationDate'] = pd.NaT

# 3. Asignar la fecha de observación para clientes ACTIVOS (Exited = 0)
# Su fecha de observación es el final del periodo de estudio.
active_mask = df_clients['Exited'] == 0
df_clients.loc[active_mask, 'ObservationDate'] = max_date

# 4. Asignar la fecha de observación para clientes que ABANDONARON (Exited = 1)
churned_mask = df_clients['Exited'] == 1

# 4a. Generar un número aleatorio de días (entre 5 y 45, por ejemplo)
n_churners = churned_mask.sum()
np.random.seed(42) # Fija la semilla para reproducibilidad (opcional)

# Genera un array de números enteros aleatorios entre 5 y 45
random_days = np.random.randint(low=5, high=46, size=n_churners)

# 4b. Convertir los días a Timedelta
random_timedelta = pd.to_timedelta(random_days, unit='D')

# 4c. Calcular la ObservationDate: ExitDate - Timedelta Aleatorio
# (Usamos .loc para asegurar que solo se aplique a los churners)
df_clients.loc[churned_mask, 'ObservationDate'] = df_clients.loc[churned_mask, 'ExitDate'] - random_timedelta

# 5. Asegurar que la columna esté en formato datetime
df_clients['ObservationDate'] = pd.to_datetime(df_clients['ObservationDate'])

print(f"✅ Columna 'ObservationDate' generada exitosamente. Total clientes: {df_clients.shape[0]}")
print("Muestra de fechas de observación para clientes que abandonaron:")
print(df_clients[churned_mask].head()[['ExitDate', 'ObservationDate']])

✅ Columna 'ObservationDate' generada exitosamente. Total clientes: 9617
Muestra de fechas de observación para clientes que abandonaron:
     ExitDate ObservationDate
0  2025-02-26      2025-01-14
2  2024-07-23      2024-06-20
5  2025-06-20      2025-06-01
7  2025-09-16      2025-09-04
16 2025-07-04      2025-06-09


In [127]:
df_clients

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label,ObservationDate
0,15634602,619,Female,42,2,0.00,1,1,1,101348.88,1,2025-02-26,Clientes potenciales,2025-01-14
1,15647311,608,Female,41,1,83807.86,1,0,1,112542.58,0,NaT,Standard,2025-09-30
2,15619304,502,Female,42,8,159660.80,3,1,0,113931.57,1,2024-07-23,VIP,2024-06-20
3,15701354,699,Female,39,1,0.00,2,0,0,93826.63,0,NaT,Nuevos - Poco Valor,2025-09-30
4,15737888,850,Female,43,2,125510.82,1,1,1,79084.10,0,NaT,Standard,2025-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,15606229,771,Male,39,5,0.00,2,1,0,96270.64,0,NaT,Clientes potenciales,2025-09-30
9613,15569892,516,Male,35,10,57369.61,1,1,1,101699.77,0,NaT,Valioso - Bajo compromiso,2025-09-30
9614,15584532,709,Female,36,7,0.00,1,0,1,42085.58,1,2025-05-16,Nuevos - Poco Valor,2025-04-23
9615,15682355,772,Male,42,3,75075.31,2,1,0,92888.52,1,2024-09-07,Standard,2024-07-31


In [122]:
df_tx.columns = ['date', 'CustomerId', 'type', 'amount']

In [123]:
df_tx['CustomerId'].nunique()

9617

In [124]:
df_tx[df_tx['type'] == 'CASH_OUT']['amount'].mean()

14048.911180581243

In [125]:
cids = df_clients['CustomerId']

In [17]:
d30_back = simulated_today_date - pd.Timedelta(days=30)
d30_back

Timestamp('2025-09-01 00:00:00')

In [18]:
ultima = df_tx['date'].max()

In [19]:
(simulated_today_date - ultima).days

1

In [20]:
cids

0       15634602
1       15647311
2       15619304
3       15701354
4       15737888
          ...   
9616    15606229
9617    15569892
9618    15584532
9619    15682355
9620    15628319
Name: CustomerId, Length: 9621, dtype: int64

In [128]:
ids = []
clients_q_total_txs = []
clients_days_since_last_tx = []
clients_total_tx_past30d = []
clients_total_tx_past90d = []
clients_avg_week_txs = []
clients_txs_total_amount = []
clients_txs_avg_amount = []
clients_avg_cashout_amount = []
clients_total_amount_past30d = []
clients_amount_std = []
clients_cashout_ratio = []
clients_transfer_ratio = []

for index, row in df_clients.iterrows():
    cid = row['CustomerId']
    obs_date = row['ObservationDate'] # 1. OBTENEMOS LA FECHA DE OBSERVACIÓN

    # 2. DEFINIMOS LAS VENTANAS DE TIEMPO
    d30_back = obs_date - pd.Timedelta(days=30)
    d90_back = obs_date - pd.Timedelta(days=90)
    
    # 3. FILTRAMOS LAS TRANSACCIONES (CRÍTICO: antes de obs_date)
    client_tx = df_tx[
        (df_tx['CustomerId'] == cid) &
        (df_tx['date'] < obs_date)
    ]

    # 4. MANEJO DE CASOS VACÍOS Y CÁLCULO DE FEATURES
    if client_tx.empty:
        # Asignar valores por defecto (0 o NaN)
        qty_txs = 0
        days_since_last_tx = (obs_date - pd.to_datetime(df_tx['date'].min())).days # Días desde el inicio
        total_tx_past30d = 0
        total_tx_past90d = 0
        avg_week_txs = 0
        total_txs_amount = 0
        avg_txs_amount = 0
        avg_cashout_amount = 0
        total_amount_past30d = 0
        amount_std = 0
        cashout_ratio = 0
        transfer_ratio = 0

    else:
        qty_txs = client_tx.shape[0]
        last_tx = client_tx['date'].max()

        # días_desde_ultima_transaccion (CORREGIDO: usa obs_date)
        days_since_last_tx = (obs_date - last_tx).days

        # Total transacciones en ventanas
        total_tx_past30d = client_tx[client_tx['date'] >= d30_back].shape[0]
        total_tx_past90d = client_tx[client_tx['date'] >= d90_back].shape[0]

        # Promedio transacciones por semana
        df_weekly = client_tx.set_index('date').resample('W').size().reset_index(name='txs_qty')
        avg_week_txs = df_weekly['txs_qty'].mean()

        # Montos
        total_txs_amount = client_tx['amount'].sum()
        avg_txs_amount = client_tx['amount'].mean()
        amount_std = client_tx['amount'].std()
        if pd.isna(amount_std): amount_std = 0

        # Features específicas
        avg_cashout_amount = client_tx[client_tx['type'] == 'CASH_OUT']['amount'].mean()
        if pd.isna(avg_cashout_amount): avg_cashout_amount = 0
        total_amount_past30d = client_tx[client_tx['date'] >= d30_back]['amount'].sum()
        
        cashout_ratio = client_tx[client_tx['type'] == 'CASH_OUT'].shape[0] / qty_txs
        transfer_ratio = client_tx[client_tx['type'] == 'TRANSFER'].shape[0] / qty_txs

    # APENDIZADO DE RESULTADOS
    ids.append(cid)
    clients_q_total_txs.append(qty_txs)
    clients_days_since_last_tx.append(days_since_last_tx)
    clients_total_tx_past30d.append(total_tx_past30d)
    clients_total_tx_past90d.append(total_tx_past90d)
    clients_avg_week_txs.append(avg_week_txs)
    clients_txs_total_amount.append(total_txs_amount)
    clients_txs_avg_amount.append(avg_txs_amount)
    clients_avg_cashout_amount.append(avg_cashout_amount)
    clients_total_amount_past30d.append(total_amount_past30d)
    clients_amount_std.append(amount_std)
    clients_cashout_ratio.append(cashout_ratio)
    clients_transfer_ratio.append(transfer_ratio)

In [21]:
"""
ids = []
clients_q_total_txs = []
clients_days_since_last_tx = []
clients_total_tx_past30d = []
clients_total_tx_past90d = []
clients_avg_week_txs = []
clients_txs_total_amount = []
clients_txs_avg_amount = []
clients_avg_cashout_amount = []
clients_total_amount_past30d = []
clients_amount_std = []
clients_cashout_ratio = []
clients_transfer_ratio = []

for cid in cids:

    client = df_tx[df_tx['CustomerId'] == cid]

    d30_back = client['ObservationDate'] - pd.Timedelta(days=30)
    d90_back = client['ObservationDate'] - pd.Timedelta(days=90)
        
    qty_txs = client.shape[0]
    last_tx = client['date'].max()
    # dias_desde_ultima_transaccion
    days_since_last_tx = (simulated_today_date - last_tx).days
    # Total transacciones ultimos 30 dias
    total_tx_past30d = client[client['date'] >= d30_back]['amount'].shape[0]
    # total_transacciones_ultimos_90_dias
    total_tx_past90d = client[client['date'] >= d90_back]['amount'].shape[0]
    # promedio_transacciones_por_semana
    df_client = client.copy()
    df_weekly = df_client.set_index('date').resample('W').size().reset_index(name='txs_qty')
    avg_week_txs = df_weekly['txs_qty'].mean()
    # monto_total_transaccionado
    total_txs_amount = client['amount'].sum()
    # monto_promedio_por_transaccion
    avg_txs_amount =  client['amount'].mean()

    
    avg_cashout_amount = client[client['type'] == 'CASH_OUT']['amount'].mean()
    if pd.isna(avg_cashout_amount):
        avg_cashout_amount = 0
    total_amount_past30d = client[client['date'] >= d30_back]['amount'].sum()
    
    amount_std = client['amount'].std()
    if pd.isna(amount_std):
        amount_std = 0
    #ratio_cash_out_vs_total_tx
    cashout_ratio = client[client['type'] == 'CASH_OUT'].shape[0] / qty_txs
    #ratio_transfer_vs_total_tx
    transfer_ratio = client[client['type'] == 'TRANSFER'].shape[0] / qty_txs


    ids.append(cid)
    clients_q_total_txs.append(qty_txs)
    clients_days_since_last_tx.append(days_since_last_tx)
    clients_total_tx_past30d.append(total_tx_past30d)
    clients_total_tx_past90d.append(total_tx_past90d)
    clients_avg_week_txs.append(avg_week_txs)
    clients_txs_total_amount.append(total_txs_amount)
    clients_txs_avg_amount.append(avg_txs_amount)
    clients_avg_cashout_amount.append(avg_cashout_amount)
    clients_total_amount_past30d.append(total_amount_past30d)
    clients_amount_std.append(amount_std)
    clients_cashout_ratio.append(cashout_ratio)
    clients_transfer_ratio.append(transfer_ratio)
"""

In [129]:
df_aggregated_transactions = pd.DataFrame({'CustomerId':  ids,
                                            'q_total_txs':  clients_q_total_txs,
                                            'days_since_last_tx':  clients_days_since_last_tx,
                                            'total_tx_past30d':  clients_total_tx_past30d,
                                            'total_tx_past90d':  clients_total_tx_past90d,
                                            'avg_week_txs':  clients_avg_week_txs,
                                            'txs_total_amount':  clients_txs_total_amount,
                                            'txs_avg_amount':  clients_txs_avg_amount,
                                            'avg_cashout_amount':  clients_avg_cashout_amount,
                                            'total_amount_past30d':  clients_total_amount_past30d,
                                            'amount_std':  clients_amount_std,
                                            'cashout_ratio': clients_cashout_ratio,
                                           'transfer_ratio': clients_transfer_ratio
                                            })

In [130]:
df_aggregated_transactions

,CustomerId,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio
0,15634602,212,1,7,26,3.028571,2100279.89,9906.980613,10103.872063,36507.52,17303.215420,0.297170,0.061321
1,15647311,107,2,6,17,1.028846,1243069.40,11617.471028,11954.757619,139483.49,18311.450829,0.392523,0.074766
2,15619304,1596,1,106,387,39.900000,32023069.02,20064.579586,19983.271760,2046816.90,23479.957418,0.548246,0.045113
3,15701354,115,1,7,20,1.074766,925832.30,8050.715652,3981.329565,32079.41,17811.222095,0.200000,0.147826
4,15737888,135,11,5,22,1.285714,1502953.07,11132.985704,10371.918679,32133.10,18922.601343,0.392593,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,15606229,348,1,23,64,3.283019,3335063.64,9583.516207,11232.376552,194568.63,16148.351675,0.250000,0.074713
9613,15569892,1256,1,78,229,11.738318,12151714.09,9674.931600,10793.962176,668268.64,16836.439986,0.270701,0.082803
9614,15584532,1040,1,31,100,12.380952,7819229.76,7518.490154,4227.181667,148628.02,14773.592372,0.242308,0.152885
9615,15682355,217,6,13,36,4.822222,2582437.20,11900.632258,11017.307639,136629.35,21255.294875,0.331797,0.064516


In [131]:
df_aggregated_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9617 entries, 0 to 9616
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CustomerId            9617 non-null   int64  
 1   q_total_txs           9617 non-null   int64  
 2   days_since_last_tx    9617 non-null   int64  
 3   total_tx_past30d      9617 non-null   int64  
 4   total_tx_past90d      9617 non-null   int64  
 5   avg_week_txs          9617 non-null   float64
 6   txs_total_amount      9617 non-null   float64
 7   txs_avg_amount        9617 non-null   float64
 8   avg_cashout_amount    9617 non-null   float64
 9   total_amount_past30d  9617 non-null   float64
 10  amount_std            9617 non-null   float64
 11  cashout_ratio         9617 non-null   float64
 12  transfer_ratio        9617 non-null   float64
dtypes: float64(8), int64(5)
memory usage: 976.9 KB


In [132]:
df_ml = df_clients.copy()
df_ml = df_ml.merge(df_aggregated_transactions, on='CustomerId')
df_ml.sample(10)

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label,ObservationDate,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio
9264,15647203,750,Female,35,3,0.00,1,1,0,191520.50,0,NaT,Nuevos - Poco Valor,2025-09-30,530,1,46,111,5.000000,4710329.10,8887.413396,4571.477143,370465.38,17583.971610,0.237736,0.141509
4747,15700601,561,Male,34,1,78829.53,1,1,1,12148.20,0,NaT,Nuevos - Poco Valor,2025-09-30,155,2,9,24,1.476190,953770.33,6153.356968,3775.810789,16861.26,12497.340759,0.245161,0.141935
2954,15697574,582,Female,40,9,0.00,3,1,1,60954.45,0,NaT,Valioso - Bajo compromiso,2025-09-30,1019,3,59,193,9.613208,9407013.34,9231.612699,10437.203106,416987.50,15780.414039,0.287537,0.086359
4071,15735270,767,Male,47,2,0.00,1,1,0,48161.18,1,2025-09-21,Clientes potenciales,2025-09-09,111,6,3,7,1.088235,714607.66,6437.906847,10188.833214,26242.41,8343.262495,0.252252,0.054054
9379,15696047,501,Male,35,6,99760.84,1,1,1,13591.52,0,NaT,Clientes potenciales,2025-09-30,818,1,47,133,7.716981,6357525.46,7772.036015,8414.744249,441462.11,13198.057056,0.284841,0.081907
5187,15733661,639,Female,27,8,133806.54,2,1,0,6251.30,0,NaT,Clientes potenciales,2025-09-30,987,1,59,182,9.224299,6597452.13,6684.348663,8011.179921,453563.41,11180.424744,0.257345,0.086120
592,15603203,650,Female,27,6,0.00,2,1,0,1002.39,0,NaT,Nuevos - Poco Valor,2025-09-30,847,1,54,144,7.915888,3108794.18,3670.359126,2537.057294,172436.89,7114.116178,0.257379,0.119244
2337,15574842,653,Female,25,2,158266.42,3,1,1,199357.24,0,NaT,VIP,2025-09-30,607,1,28,97,5.726415,11885433.66,19580.615585,21355.635373,408844.12,22397.698235,0.551895,0.031301
8814,15612103,627,Female,35,2,137852.96,1,1,1,172269.21,1,2025-05-19,Standard,2025-05-14,148,20,1,8,1.761905,1597400.48,10793.246486,12383.423878,7978.51,17542.691722,0.331081,0.081081
7005,15611973,804,Male,55,7,0.00,2,1,1,118752.60,0,NaT,Valioso - Bajo compromiso,2025-09-30,654,1,37,113,6.112150,7639523.84,11681.229113,12374.142826,519974.03,20003.818237,0.281346,0.094801


In [133]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033175 entries, 0 to 1033174
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   session_id     1033175 non-null  object        
 1   CustomerId     1033175 non-null  int64         
 2   cluster_label  1033175 non-null  object        
 3   date           1033175 non-null  datetime64[ns]
 4   duration_min   1033175 non-null  float64       
 5   used_transfer  1033175 non-null  int32         
 6   used_payment   1033175 non-null  int32         
 7   used_invest    1033175 non-null  int32         
 8   opened_push    1033175 non-null  int32         
 9   failed_login   1033175 non-null  int32         
 10  month          1033175 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int32(5), int64(1), object(2)
memory usage: 67.0+ MB


In [134]:
df_sessions.sample(10)

,session_id,CustomerId,cluster_label,date,duration_min,used_transfer,used_payment,used_invest,opened_push,failed_login,month
584349,598736,15662232,Standard,2024-09-07,17.905202,0,0,0,0,0,2024-09-01
55528,56858,15653849,Standard,2025-01-20,8.038960,1,0,0,0,0,2025-01-01
461933,472538,15724099,Standard,2023-11-25,19.349191,0,0,0,0,0,2023-11-01
597674,612293,15663168,Standard,2025-02-02,35.370865,0,0,0,0,0,2025-02-01
36194,37111,15625461,VIP,2025-08-20,33.482493,0,0,1,0,0,2025-08-01
83734,85555,15756026,Nuevos - Poco Valor,2024-03-15,18.218110,0,0,0,0,0,2024-03-01
232652,237563,15736069,Standard,2025-01-14,5.335163,1,1,0,0,0,2025-01-01
962204,987048,15575486,Nuevos - Poco Valor,2025-07-05,8.089226,0,0,0,1,0,2025-07-01
678901,696230,15705313,Nuevos - Poco Valor,2024-06-22,4.390688,0,0,0,0,0,2024-06-01
522920,535720,15619238,Standard,2024-07-09,31.592105,0,0,0,0,0,2024-07-01


In [135]:
import pandas as pd

# Inicialización de listas (las mantengo como las definiste)
ids = []
clients_q_total_ss = []
clients_days_since_last_ss = []
clients_total_ss_past30d = []
clients_total_ss_past90d = []
clients_avg_week_ss = []
clients_ss_total_time = []
clients_ss_avg_time = []
clients_ss_std_time = []
clients_total_used_transfer = []
clients_total_used_payment = []
clients_total_used_invest = []
clients_total_open_push = []
clients_total_failed_ss = []

# Iteramos sobre df_clients para obtener la ObservationDate individual
for index, row in df_clients.iterrows():
    cid = row['CustomerId']
    obs_date = row['ObservationDate'] # OBTENEMOS LA FECHA DE CORTE INDIVIDUAL

    # 1. DEFINIMOS LAS VENTANAS DE TIEMPO usando la ObservationDate
    d30_back = obs_date - pd.Timedelta(days=30)
    d90_back = obs_date - pd.Timedelta(days=90)

    # 2. FILTRAMOS LAS SESIONES (CRÍTICO: por Cliente Y antes de la ObservationDate)
    client_ss = df_sessions[
        (df_sessions['CustomerId'] == cid) &
        (df_sessions['date'] < obs_date)
    ]

    # 3. MANEJO DE CASOS VACÍOS Y CÁLCULO DE FEATURES
    if client_ss.empty:
        # Asignar 0 o un valor por defecto para clientes sin actividad en la ventana de observación
        q_total_ss = 0
        days_since_last_ss = (obs_date - pd.to_datetime(df_sessions['date'].min())).days 
        total_ss_past30d = 0
        total_ss_past90d = 0
        avg_week_ss = 0
        total_ss_time = 0
        avg_ss_time = 0
        std_ss_time = 0
        total_used_transfers = 0
        total_used_payments = 0
        total_used_invest = 0
        total_opened_push = 0
        total_failed_ss = 0
    else:
        q_total_ss = client_ss.shape[0]
        last_ss = client_ss['date'].max()

        # días_desde_ultima_sesión (CORREGIDO: usa obs_date)
        days_since_last_ss = (obs_date - last_ss).days

        # Total sesiones en ventanas temporales
        total_ss_past30d = client_ss[client_ss['date'] >= d30_back].shape[0]
        total_ss_past90d = client_ss[client_ss['date'] >= d90_back].shape[0]

        # Promedio sesiones por semana
        df_weekly = client_ss.set_index('date').resample('W').size().reset_index(name='ss_qty')
        avg_week_ss = df_weekly['ss_qty'].mean()

        # Tiempos de sesión
        total_ss_time = client_ss['duration_min'].sum()
        avg_ss_time = client_ss['duration_min'].mean()
        std_ss_time = client_ss['duration_min'].std()
        if pd.isna(std_ss_time): std_ss_time = 0

        # Totales de eventos
        total_used_transfers = client_ss['used_transfer'].sum()
        total_used_payments = client_ss['used_payment'].sum()
        total_used_invest = client_ss['used_invest'].sum()
        total_opened_push = client_ss['opened_push'].sum()
        total_failed_ss = client_ss['failed_login'].sum()

    # APENDIZADO DE RESULTADOS
    ids.append(cid)
    clients_q_total_ss.append(q_total_ss)
    clients_days_since_last_ss.append(days_since_last_ss)
    clients_total_ss_past30d.append(total_ss_past30d)
    clients_total_ss_past90d.append(total_ss_past90d)
    clients_avg_week_ss.append(avg_week_ss)
    clients_ss_total_time.append(total_ss_time)
    clients_ss_avg_time.append(avg_ss_time)
    clients_ss_std_time.append(std_ss_time)
    clients_total_used_transfer.append(total_used_transfers)
    clients_total_used_payment.append(total_used_payments)
    clients_total_used_invest.append(total_used_invest)
    clients_total_open_push.append(total_opened_push)
    clients_total_failed_ss.append(total_failed_ss)

In [28]:
"""
ids = []
clients_q_total_ss = []
clients_days_since_last_ss = []
clients_total_ss_past30d = []
clients_total_ss_past90d = []
clients_avg_week_ss = []
clients_ss_total_time = []
clients_ss_avg_time = []
clients_ss_std_time = []
clients_total_used_transfer = []
clients_total_used_payment = []
clients_total_used_invest = []
clients_total_open_push = []
clients_total_failed_ss = []

d30_back = simulated_today_date - pd.Timedelta(days=30)
d90_back = simulated_today_date - pd.Timedelta(days=90)

for cid in cids:
    
    client = df_sessions[df_sessions['CustomerId'] == cid]

        
    q_total_ss = client.shape[0]
    last_ss = client['date'].max()
    # dias_desde_ultima_transaccion
    days_since_last_ss = (simulated_today_date - last_ss).days
    # Total transacciones ultimos 30 dias
    total_ss_past30d = client[client['date'] >= d30_back].shape[0]
    # total_transacciones_ultimos_90_dias
    total_ss_past90d = client[client['date'] >= d90_back].shape[0]
    # promedio_transacciones_por_semana
    df_client = client.copy()
    df_weekly = df_client.set_index('date').resample('W').size().reset_index(name='ss_qty')
    avg_week_ss = df_weekly['ss_qty'].mean()
    # monto_total_transaccionado
    total_ss_time = client['duration_min'].sum()
    # monto_promedio_por_transaccion
    avg_ss_time = client['duration_min'].mean()
    # std tiempo uso
    std_ss_time = client['duration_min'].std()
    # cantidad de trasnferencias
    total_used_transfers = client['used_transfer'].sum()
    # cantidad de payments
    total_used_payments = client['used_payment'].sum()
    # cantidad de investments
    total_used_invest = client['used_invest'].sum()
    # total notificaciones abiertas
    total_opened_push = client['opened_push'].sum()
    # cantidad de logins fallidos
    total_failed_ss = client['failed_login'].sum()


    ids.append(cid)
    clients_q_total_ss.append(q_total_ss)
    clients_days_since_last_ss.append(days_since_last_ss)
    clients_total_ss_past30d.append(total_ss_past30d)
    clients_total_ss_past90d.append(total_ss_past90d)
    clients_avg_week_ss.append(avg_week_ss)
    clients_ss_total_time.append(total_ss_time)
    clients_ss_avg_time.append(avg_ss_time)
    clients_ss_std_time.append(std_ss_time)
    clients_total_used_transfer.append(total_used_transfers)
    clients_total_used_payment.append(total_used_payments)
    clients_total_used_invest.append(total_used_invest)
    clients_total_open_push.append(total_opened_push)
    clients_total_failed_ss.append(total_failed_ss)
"""

In [136]:
df_aggregated_sessions = pd.DataFrame({'CustomerId': ids,
                                       'total_ss': clients_q_total_ss,
                                       'days_since_last_ss': clients_days_since_last_ss,
                                       'total_ss_past30d': clients_total_ss_past30d,
                                       'total_ss_past90d': clients_total_ss_past90d,
                                       'avg_ss_per_wk': clients_avg_week_ss,
                                       'total_ss_duration_min': clients_ss_total_time,
                                       'avg_ss_duration_min': clients_ss_avg_time,
                                       'std_ss_duration_min': clients_ss_std_time,
                                       'total_used_transfer': clients_total_used_transfer,
                                       'total_used_payment': clients_total_used_payment,
                                       'total_used_invest': clients_total_used_invest,
                                       'total_opened_push': clients_total_open_push,
                                       'total_failed_ss': clients_total_failed_ss})

In [137]:
df_aggregated_sessions

,CustomerId,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss
0,15634602,52,5,1,6,0.732394,708.292943,13.621018,20.269334,5,2,1,7,0
1,15647311,109,3,6,12,1.009259,2084.057633,19.119795,17.111199,9,18,1,22,1
2,15619304,75,6,2,6,1.785714,2109.397254,28.125297,28.488867,16,14,11,37,0
3,15701354,32,10,1,3,0.307692,256.659108,8.020597,4.292557,0,0,0,3,0
4,15737888,109,1,5,11,1.028302,2351.063602,21.569391,24.417880,13,17,3,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,15606229,111,6,3,17,1.018349,1362.327477,12.273221,9.288054,9,10,5,18,0
9613,15569892,138,14,4,19,1.289720,2679.499070,19.416660,16.945033,18,6,3,21,4
9614,15584532,45,71,0,1,0.633803,369.968901,8.221531,5.570309,1,1,0,3,1
9615,15682355,35,45,0,5,0.833333,688.892454,19.682642,18.304509,2,1,0,5,1


In [138]:
df_ml = df_ml.merge(df_aggregated_sessions, on='CustomerId')

In [139]:
df_ml.sample(10)

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label,ObservationDate,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss
393,15701376,668,Male,37,10,152958.29,2,1,1,159585.61,0,NaT,VIP,2025-09-30,3811,1,216,653,35.616822,79683675.45,20908.862621,20593.673040,4267919.41,24851.454040,0.542902,0.048019,326,3,7,35,2.990826,9993.198767,30.653984,28.067464,78,81,31,120,3
4592,15761506,615,Male,19,5,0.00,2,1,0,159920.92,0,NaT,Clientes potenciales,2025-09-30,736,1,32,110,6.878505,7809989.35,10611.398573,12077.357638,275722.67,18277.686189,0.270380,0.092391,102,10,6,18,0.962264,1190.445573,11.671035,8.142154,11,5,3,12,4
6956,15707602,539,Female,47,2,127286.04,2,1,1,166929.43,1,2025-09-11,VIP,2025-07-31,551,1,42,110,5.622449,12873733.50,23364.307623,20650.791466,870022.25,27747.371665,0.557169,0.068966,178,28,1,12,1.854167,5692.745393,31.981716,31.125759,45,37,22,61,1
2336,15783305,593,Female,46,7,98752.51,1,1,0,145560.38,0,NaT,Valioso - Bajo compromiso,2025-09-30,583,3,27,96,5.500000,5763017.84,9885.107787,12375.935782,265887.16,16663.551342,0.252144,0.072041,125,1,6,21,1.136364,2232.285817,17.858287,17.438660,14,18,3,16,3
7339,15591107,723,Female,68,3,110357.00,1,0,0,141977.54,1,2025-05-04,Standard,2025-03-29,138,21,2,8,1.792208,1711280.44,12400.582899,11254.515745,8820.93,15947.795379,0.340580,0.108696,78,4,3,12,0.962963,1603.030281,20.551670,23.427778,11,10,2,12,3
5610,15771270,635,Female,27,8,127471.56,1,1,1,152916.05,1,2025-07-11,Standard,2025-06-03,435,1,13,36,4.833333,5385308.87,12380.020391,11908.109795,229191.47,20567.657950,0.335632,0.087356,103,10,1,4,1.144444,2152.467982,20.897747,16.799949,13,11,0,20,1
7286,15687634,561,Male,49,5,94754.00,1,1,1,26691.31,0,NaT,Standard,2025-09-30,345,2,17,60,3.254717,3327067.64,9643.674319,10066.561532,88752.33,15006.358673,0.321739,0.098551,115,1,5,17,1.045455,2197.832697,19.111589,16.971250,10,11,2,23,4
4952,15680597,784,Male,38,1,138515.02,1,1,1,171768.76,0,NaT,Standard,2025-09-30,106,1,6,21,1.019231,1181063.95,11142.112736,10105.609737,33226.96,19341.915290,0.358491,0.075472,121,3,5,13,1.110092,2243.204305,18.538879,19.834638,16,12,2,27,2
163,15588537,615,Female,41,9,109013.23,1,1,0,196499.96,0,NaT,Valioso - Bajo compromiso,2025-09-30,1042,1,66,168,9.738318,12225996.31,11733.201833,13755.888955,1009706.82,20791.258187,0.275432,0.079655,131,4,8,18,1.201835,2598.752507,19.837805,17.685183,12,14,2,19,3
4646,15799357,727,Male,35,5,136364.46,1,0,0,142754.71,0,NaT,Standard,2025-09-30,426,2,28,76,4.018868,4630174.65,10868.954577,11424.259254,452494.77,16459.231710,0.314554,0.077465,113,4,1,5,1.036697,2253.167691,19.939537,15.258213,10,14,6,19,3


In [140]:
pd.set_option('display.max_columns', None)
df_ml[df_ml['CustomerId'] == 15565701]

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label,ObservationDate,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss
1243,15565701,698,Female,39,9,161993.89,1,0,0,90212.38,0,NaT,Valioso - Bajo compromiso,2025-09-30,958,1,63,163,8.953271,9722750.4,10149.008768,9771.369663,445972.75,17796.817289,0.278706,0.08142,112,4,4,13,1.037037,2305.550628,20.585273,19.218159,14,16,4,10,3


In [141]:
df_cl_app_features

,CustomerId,total_sessions,mean_sessions_per_month,median_session_duration,pct_sessions_with_transfer,total_failed_logins,months_active,last_session_date,cluster_label,Exited,Tenure
0,15565701,113,4.520000,18.043798,0.100683,3,25,2025-09-30,Valioso - Bajo compromiso,0,9
1,15565706,33,1.833333,7.847351,0.000000,0,18,2025-06-06,Nuevos - Poco Valor,1,1
2,15565714,109,4.360000,20.580506,0.138143,0,25,2025-09-19,Standard,0,1
3,15565779,130,5.652174,13.791449,0.060197,1,23,2025-09-27,Clientes potenciales,0,6
4,15565796,142,5.680000,17.602444,0.084667,2,25,2025-09-30,Standard,0,10
...,...,...,...,...,...,...,...,...,...,...,...
9612,15815628,105,4.375000,11.526779,0.056415,2,24,2025-08-25,Clientes potenciales,0,8
9613,15815645,281,11.240000,29.469579,0.233503,3,25,2025-09-30,VIP,0,8
9614,15815656,237,11.285714,29.198323,0.253753,2,21,2025-05-29,VIP,1,9
9615,15815660,85,3.541667,12.054533,0.021230,1,24,2025-09-28,Clientes potenciales,0,1


In [142]:
df_cl_app_features = df_cl_app_features.drop(['total_sessions', 'total_failed_logins', 'months_active', 
                                              'Exited', 'Tenure', 'cluster_label','last_session_date'], axis=1)

In [143]:
df_cl_app_features

,CustomerId,mean_sessions_per_month,median_session_duration,pct_sessions_with_transfer
0,15565701,4.520000,18.043798,0.100683
1,15565706,1.833333,7.847351,0.000000
2,15565714,4.360000,20.580506,0.138143
3,15565779,5.652174,13.791449,0.060197
4,15565796,5.680000,17.602444,0.084667
...,...,...,...,...
9612,15815628,4.375000,11.526779,0.056415
9613,15815645,11.240000,29.469579,0.233503
9614,15815656,11.285714,29.198323,0.253753
9615,15815660,3.541667,12.054533,0.021230


In [144]:
df_ml = df_ml.merge(df_cl_app_features, on='CustomerId')

In [145]:
df_ml.sample(10)

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,ExitDate,cluster_label,ObservationDate,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss,mean_sessions_per_month,median_session_duration,pct_sessions_with_transfer
1216,15743532,704,Male,27,5,147004.34,1,1,0,64381.33,1,2025-08-28,Clientes potenciales,2025-07-20,480,7,11,36,5.052632,4524822.90,9426.714375,10026.354344,61323.96,16298.672981,0.254167,0.085417,66,33,0,2,0.694737,1023.320957,15.504863,17.379337,6,7,1,13,2,3.142857,13.114012,0.065476
1377,15710206,591,Female,39,4,150500.64,1,1,0,14928.80,0,NaT,Clientes potenciales,2025-09-30,306,1,12,49,2.859813,2296331.64,7504.351765,8396.800353,47754.25,12218.456898,0.277778,0.078431,85,2,7,11,0.794393,1235.885180,14.539826,8.600817,4,5,1,7,2,3.583333,14.307229,0.034722
9589,15747927,758,Male,26,4,155739.76,1,1,0,171552.02,0,NaT,Standard,2025-09-30,307,1,16,59,2.869159,3373129.65,10987.392997,13664.641068,158230.96,19047.184438,0.335505,0.065147,121,3,9,13,1.110092,2218.693408,18.336309,17.343053,15,9,4,15,3,5.041667,17.004677,0.122652
9106,15592104,655,Female,41,5,0.00,1,0,0,36548.00,1,2025-08-08,Nuevos - Poco Valor,2025-07-21,568,1,13,43,5.916667,3167546.67,5576.666673,4503.737673,67345.62,9995.646269,0.279930,0.109155,49,59,0,5,0.550562,445.760725,9.097158,5.005394,2,4,0,7,0,2.882353,8.829768,0.034314
3671,15670562,470,Male,30,3,101140.76,1,1,1,50906.65,0,NaT,Clientes potenciales,2025-09-30,292,2,16,51,2.754717,2616221.14,8959.661438,8327.112785,129322.82,14964.442441,0.270548,0.082192,91,10,3,8,0.842593,1201.228352,13.200312,10.643835,3,12,1,12,3,3.791667,12.303932,0.031250
6992,15692718,738,Female,38,7,0.00,2,0,0,69227.42,0,NaT,Clientes potenciales,2025-09-30,973,1,56,174,9.093458,8954259.57,9202.733371,10354.046399,350443.85,15185.154440,0.293936,0.090442,86,9,6,12,0.803738,1073.443593,12.481902,10.654916,6,7,0,10,1,3.739130,11.197944,0.089130
1386,15596021,598,Male,44,8,0.00,2,1,0,148487.90,0,NaT,Valioso - Bajo compromiso,2025-09-30,1290,1,84,234,12.056075,13720988.53,10636.425217,11599.416544,868113.41,19080.445497,0.273643,0.082946,124,4,7,19,1.137615,2068.503219,16.681478,12.880119,13,14,9,17,0,5.208333,16.384620,0.108116
5468,15795132,735,Female,25,3,91718.80,1,0,0,28411.23,0,NaT,Nuevos - Poco Valor,2025-09-30,377,3,24,66,3.556604,2488548.32,6600.923926,3828.120680,132597.91,12362.307160,0.273210,0.156499,41,6,1,8,0.379630,374.500156,9.134150,5.523803,2,2,2,4,0,1.863636,8.264364,0.031818
733,15815364,736,Female,28,2,0.00,2,1,1,117431.10,0,NaT,Clientes potenciales,2025-09-30,235,5,18,46,2.281553,2526773.39,10752.227191,12916.672642,114776.05,16816.018014,0.225532,0.097872,85,15,4,8,0.787037,1010.171255,11.884368,9.655365,3,8,1,14,1,3.400000,11.108114,0.038000
3086,15736397,544,Male,23,1,96471.20,1,1,0,35550.97,0,NaT,Nuevos - Poco Valor,2025-09-30,181,2,10,33,1.707547,1228704.10,6788.420442,3675.702895,113007.35,13927.363090,0.209945,0.132597,38,14,1,9,0.372549,331.896113,8.734108,6.324623,1,3,1,1,0,2.111111,7.810511,0.013889


In [146]:
columnas = ['CreditScore', 'Age', 'Tenure', 'Balance',
           'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'q_total_txs',
           'days_since_last_tx', 'total_tx_past30d', 'total_tx_past90d',
           'avg_week_txs', 'txs_total_amount', 'txs_avg_amount',
           'avg_cashout_amount', 'total_amount_past30d', 'amount_std',
           'cashout_ratio', 'transfer_ratio', 'total_ss', 'days_since_last_ss',
           'total_ss_past30d', 'total_ss_past90d', 'avg_ss_per_wk',
           'total_ss_duration_min', 'avg_ss_duration_min', 'std_ss_duration_min',
           'total_used_transfer', 'total_used_payment', 'total_used_invest',
           'total_opened_push', 'total_failed_ss', 'mean_sessions_per_month',
           'median_session_duration', 'pct_sessions_with_transfer',
           'Exited']

df_ml = df_ml[columnas]
df_ml

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss,mean_sessions_per_month,median_session_duration,pct_sessions_with_transfer,Exited
0,619,42,2,0.00,1,1,1,101348.88,212,1,7,26,3.028571,2100279.89,9906.980613,10103.872063,36507.52,17303.215420,0.297170,0.061321,52,5,1,6,0.732394,708.292943,13.621018,20.269334,5,2,1,7,0,3.312500,9.572620,0.130208,1
1,608,41,1,83807.86,1,0,1,112542.58,107,2,6,17,1.028846,1243069.40,11617.471028,11954.757619,139483.49,18311.450829,0.392523,0.074766,109,3,6,12,1.009259,2084.057633,19.119795,17.111199,9,18,1,22,1,4.360000,17.284281,0.082000,0
2,502,42,8,159660.80,3,1,0,113931.57,1596,1,106,387,39.900000,32023069.02,20064.579586,19983.271760,2046816.90,23479.957418,0.548246,0.045113,75,6,2,6,1.785714,2109.397254,28.125297,28.488867,16,14,11,37,0,9.750000,24.654271,0.273571,1
3,699,39,1,0.00,2,0,0,93826.63,115,1,7,20,1.074766,925832.30,8050.715652,3981.329565,32079.41,17811.222095,0.200000,0.147826,32,10,1,3,0.307692,256.659108,8.020597,4.292557,0,0,0,3,0,1.600000,7.874464,0.000000,0
4,850,43,2,125510.82,1,1,1,79084.10,135,11,5,22,1.285714,1502953.07,11132.985704,10371.918679,32133.10,18922.601343,0.392593,0.066667,109,1,5,11,1.028302,2351.063602,21.569391,24.417880,13,17,3,15,1,4.360000,21.391359,0.154762,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,771,39,5,0.00,2,1,0,96270.64,348,1,23,64,3.283019,3335063.64,9583.516207,11232.376552,194568.63,16148.351675,0.250000,0.074713,111,6,3,17,1.018349,1362.327477,12.273221,9.288054,9,10,5,18,0,4.440000,11.714599,0.069762,0
9613,516,35,10,57369.61,1,1,1,101699.77,1256,1,78,229,11.738318,12151714.09,9674.931600,10793.962176,668268.64,16836.439986,0.270701,0.082803,138,14,4,19,1.289720,2679.499070,19.416660,16.945033,18,6,3,21,4,5.520000,19.511152,0.139501,0
9614,709,36,7,0.00,1,0,1,42085.58,1040,1,31,100,12.380952,7819229.76,7518.490154,4227.181667,148628.02,14773.592372,0.242308,0.152885,45,71,0,1,0.633803,369.968901,8.221531,5.570309,1,1,0,3,1,3.000000,8.114665,0.016667,1
9615,772,42,3,75075.31,2,1,0,92888.52,217,6,13,36,4.822222,2582437.20,11900.632258,11017.307639,136629.35,21255.294875,0.331797,0.064516,35,45,0,5,0.833333,688.892454,19.682642,18.304509,2,1,0,5,1,3.500000,15.708641,0.037500,1


In [147]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.preprocessing import RobustScaler

In [148]:
vif_robust_scaler = RobustScaler()

df_vif = df_ml.copy()
df_vif.drop(['Exited'], axis=1, inplace=True)

In [149]:
columnas_df_vif = df_vif.columns
len(columnas_df_vif)

36

In [150]:
X_vif = vif_robust_scaler.fit_transform(df_vif)
df_vif[columnas_df_vif] = X_vif
df_vif

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,q_total_txs,days_since_last_tx,total_tx_past30d,total_tx_past90d,avg_week_txs,txs_total_amount,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,total_ss,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_per_wk,total_ss_duration_min,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss,mean_sessions_per_month,median_session_duration,pct_sessions_with_transfer
0,-0.246269,0.416667,-0.6,-0.761000,0.0,0.0,0.0,0.014671,-0.575188,0.0,-0.628571,-0.602041,-0.411805,-0.557699,-0.068067,-0.298583,-0.651894,0.046619,0.194788,-0.976240,-1.085106,-0.166667,-0.50,-0.625,-0.661593,-0.735554,-0.521897,0.553888,-0.4,-0.888889,-0.50,-0.727273,-1.0,-0.625000,-0.863524,0.576429
1,-0.328358,0.333333,-0.8,-0.103956,0.0,-1.0,0.0,0.128585,-0.772556,1.0,-0.657143,-0.693878,-0.803524,-0.717358,0.454195,0.328382,-0.369798,0.234494,1.409500,-0.464216,0.127660,-0.333333,0.75,0.125,0.047191,0.220824,0.164324,0.234688,0.0,0.888889,-0.50,0.636364,-0.5,0.016327,0.177296,-0.069338
2,-1.119403,0.416667,0.6,0.490723,2.0,0.0,-1.0,0.142720,2.026316,0.0,2.200000,3.081633,6.810794,5.015542,3.033341,3.047944,4.855215,1.197598,3.393251,-1.593459,-0.595745,-0.083333,-0.25,-0.625,2.034943,0.238439,1.288167,1.384656,0.7,0.444444,2.00,2.000000,-1.0,3.316327,1.172002,2.496816
3,0.350746,0.166667,-0.8,-0.761000,1.0,-1.0,-1.0,-0.061881,-0.757519,0.0,-0.628571,-0.663265,-0.794529,-0.776445,-0.634838,-2.372520,-0.664025,0.141281,-1.043062,2.317985,-1.510638,0.250000,-0.50,-1.000,-1.748846,-1.049512,-1.220803,-1.060922,-0.9,-1.111111,-0.75,-1.090909,-1.0,-1.673469,-1.092719,-1.167757
4,1.477612,0.500000,-0.6,0.222991,0.0,0.0,0.0,-0.211910,-0.719925,10.0,-0.685714,-0.642857,-0.753207,-0.668954,0.306268,-0.207785,-0.663878,0.348377,1.410382,-0.772662,0.127660,-0.500000,0.50,0.000,0.095940,0.406436,0.470021,0.973192,0.4,0.777778,0.00,0.000000,-0.5,0.016327,0.731616,0.905333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,0.888060,0.166667,0.0,-0.761000,1.0,0.0,-1.0,-0.037009,-0.319549,0.0,-0.171429,-0.214286,-0.361963,-0.327716,-0.166830,0.083685,-0.218896,-0.168579,-0.406110,-0.466261,0.170213,-0.083333,0.00,0.750,0.070460,-0.280894,-0.690096,-0.556015,0.0,0.000000,0.50,0.272727,-1.0,0.065306,-0.574427,-0.233272
9613,-1.014925,-0.166667,1.0,-0.311228,0.0,0.0,0.0,0.018241,1.387218,0.0,1.400000,1.469388,1.294313,1.314421,-0.138918,-0.064823,1.078773,-0.040360,-0.142404,-0.158188,0.744681,0.583333,0.25,1.000,0.765179,0.634752,0.201371,0.217894,0.9,-0.444444,0.00,0.545455,1.0,0.726531,0.477850,0.700903
9614,0.425373,-0.083333,0.4,-0.761000,0.0,-1.0,0.0,-0.588431,0.981203,0.0,0.057143,0.153061,1.420196,0.507478,-0.797342,-2.289240,-0.344748,-0.424753,-0.504103,2.510620,-1.234043,5.333333,-0.75,-1.250,-0.913991,-0.970744,-1.195727,-0.931776,-0.8,-1.000000,-0.75,-1.090909,-0.5,-0.816327,-1.060299,-0.944501
9615,0.895522,0.416667,-0.4,-0.172418,1.0,0.0,-1.0,-0.071427,-0.565789,5.0,-0.457143,-0.500000,-0.060454,-0.467896,0.540653,0.010833,-0.377617,0.783053,0.635908,-0.854557,-1.446809,3.166667,-0.75,-0.750,-0.403186,-0.749040,0.234564,0.355299,-0.7,-1.000000,-0.75,-0.909091,-0.5,-0.510204,-0.035363,-0.665431


In [151]:
len(columnas_df_vif)

36

In [152]:
df_vif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9617 entries, 0 to 9616
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 9617 non-null   float64
 1   Age                         9617 non-null   float64
 2   Tenure                      9617 non-null   float64
 3   Balance                     9617 non-null   float64
 4   NumOfProducts               9617 non-null   float64
 5   HasCrCard                   9617 non-null   float64
 6   IsActiveMember              9617 non-null   float64
 7   EstimatedSalary             9617 non-null   float64
 8   q_total_txs                 9617 non-null   float64
 9   days_since_last_tx          9617 non-null   float64
 10  total_tx_past30d            9617 non-null   float64
 11  total_tx_past90d            9617 non-null   float64
 12  avg_week_txs                9617 non-null   float64
 13  txs_total_amount            9617 

In [153]:
features = columnas_df_vif

In [154]:
df_vif = df_vif[features].fillna(0)

In [155]:
df_vif_1 = pd.DataFrame()
df_vif_1['Feature'] = features
df_vif_1['VIF'] = [vif(df_vif[features], i) for i in range(len(features))]

In [156]:
df_vif_1.sort_values('VIF', ascending=False)

,Feature,VIF
25,total_ss_duration_min,117.880757
24,avg_ss_per_wk,92.696735
33,mean_sessions_per_month,83.264095
13,txs_total_amount,61.068895
8,q_total_txs,53.897005
20,total_ss,45.201408
10,total_tx_past30d,40.843192
26,avg_ss_duration_min,38.717785
28,total_used_transfer,36.518344
16,total_amount_past30d,33.634092


In [157]:
selected_vif = df_vif_1[df_vif_1['VIF'] < 40]
len(selected_vif)

29

In [158]:
selected_features = selected_vif['Feature'].values
selected_features = selected_features.tolist()

In [159]:
selected_features.append('Exited')

In [160]:
len(selected_features)

30

In [161]:
df_ml_vif = df_ml[selected_features].copy()

In [162]:
df_ml_vif

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,days_since_last_tx,total_tx_past90d,avg_week_txs,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss,median_session_duration,pct_sessions_with_transfer,Exited
0,619,42,2,0.00,1,1,1,101348.88,1,26,3.028571,9906.980613,10103.872063,36507.52,17303.215420,0.297170,0.061321,5,1,6,13.621018,20.269334,5,2,1,7,0,9.572620,0.130208,1
1,608,41,1,83807.86,1,0,1,112542.58,2,17,1.028846,11617.471028,11954.757619,139483.49,18311.450829,0.392523,0.074766,3,6,12,19.119795,17.111199,9,18,1,22,1,17.284281,0.082000,0
2,502,42,8,159660.80,3,1,0,113931.57,1,387,39.900000,20064.579586,19983.271760,2046816.90,23479.957418,0.548246,0.045113,6,2,6,28.125297,28.488867,16,14,11,37,0,24.654271,0.273571,1
3,699,39,1,0.00,2,0,0,93826.63,1,20,1.074766,8050.715652,3981.329565,32079.41,17811.222095,0.200000,0.147826,10,1,3,8.020597,4.292557,0,0,0,3,0,7.874464,0.000000,0
4,850,43,2,125510.82,1,1,1,79084.10,11,22,1.285714,11132.985704,10371.918679,32133.10,18922.601343,0.392593,0.066667,1,5,11,21.569391,24.417880,13,17,3,15,1,21.391359,0.154762,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,771,39,5,0.00,2,1,0,96270.64,1,64,3.283019,9583.516207,11232.376552,194568.63,16148.351675,0.250000,0.074713,6,3,17,12.273221,9.288054,9,10,5,18,0,11.714599,0.069762,0
9613,516,35,10,57369.61,1,1,1,101699.77,1,229,11.738318,9674.931600,10793.962176,668268.64,16836.439986,0.270701,0.082803,14,4,19,19.416660,16.945033,18,6,3,21,4,19.511152,0.139501,0
9614,709,36,7,0.00,1,0,1,42085.58,1,100,12.380952,7518.490154,4227.181667,148628.02,14773.592372,0.242308,0.152885,71,0,1,8.221531,5.570309,1,1,0,3,1,8.114665,0.016667,1
9615,772,42,3,75075.31,2,1,0,92888.52,6,36,4.822222,11900.632258,11017.307639,136629.35,21255.294875,0.331797,0.064516,45,0,5,19.682642,18.304509,2,1,0,5,1,15.708641,0.037500,1


In [163]:
vif_robust_scaler = RobustScaler()

df_vif = df_ml_vif.copy()
df_vif.drop(['Exited'], axis=1, inplace=True)

In [164]:
columnas_df_vif = df_vif.columns
len(columnas_df_vif)

29

In [165]:
X_vif = vif_robust_scaler.fit_transform(df_vif)
df_vif[columnas_df_vif] = X_vif
df_vif

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,days_since_last_tx,total_tx_past90d,avg_week_txs,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,days_since_last_ss,total_ss_past30d,total_ss_past90d,avg_ss_duration_min,std_ss_duration_min,total_used_transfer,total_used_payment,total_used_invest,total_opened_push,total_failed_ss,median_session_duration,pct_sessions_with_transfer
0,-0.246269,0.416667,-0.6,-0.761000,0.0,0.0,0.0,0.014671,0.0,-0.602041,-0.411805,-0.068067,-0.298583,-0.651894,0.046619,0.194788,-0.976240,-0.166667,-0.50,-0.625,-0.521897,0.553888,-0.4,-0.888889,-0.50,-0.727273,-1.0,-0.863524,0.576429
1,-0.328358,0.333333,-0.8,-0.103956,0.0,-1.0,0.0,0.128585,1.0,-0.693878,-0.803524,0.454195,0.328382,-0.369798,0.234494,1.409500,-0.464216,-0.333333,0.75,0.125,0.164324,0.234688,0.0,0.888889,-0.50,0.636364,-0.5,0.177296,-0.069338
2,-1.119403,0.416667,0.6,0.490723,2.0,0.0,-1.0,0.142720,0.0,3.081633,6.810794,3.033341,3.047944,4.855215,1.197598,3.393251,-1.593459,-0.083333,-0.25,-0.625,1.288167,1.384656,0.7,0.444444,2.00,2.000000,-1.0,1.172002,2.496816
3,0.350746,0.166667,-0.8,-0.761000,1.0,-1.0,-1.0,-0.061881,0.0,-0.663265,-0.794529,-0.634838,-2.372520,-0.664025,0.141281,-1.043062,2.317985,0.250000,-0.50,-1.000,-1.220803,-1.060922,-0.9,-1.111111,-0.75,-1.090909,-1.0,-1.092719,-1.167757
4,1.477612,0.500000,-0.6,0.222991,0.0,0.0,0.0,-0.211910,10.0,-0.642857,-0.753207,0.306268,-0.207785,-0.663878,0.348377,1.410382,-0.772662,-0.500000,0.50,0.000,0.470021,0.973192,0.4,0.777778,0.00,0.000000,-0.5,0.731616,0.905333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9612,0.888060,0.166667,0.0,-0.761000,1.0,0.0,-1.0,-0.037009,0.0,-0.214286,-0.361963,-0.166830,0.083685,-0.218896,-0.168579,-0.406110,-0.466261,-0.083333,0.00,0.750,-0.690096,-0.556015,0.0,0.000000,0.50,0.272727,-1.0,-0.574427,-0.233272
9613,-1.014925,-0.166667,1.0,-0.311228,0.0,0.0,0.0,0.018241,0.0,1.469388,1.294313,-0.138918,-0.064823,1.078773,-0.040360,-0.142404,-0.158188,0.583333,0.25,1.000,0.201371,0.217894,0.9,-0.444444,0.00,0.545455,1.0,0.477850,0.700903
9614,0.425373,-0.083333,0.4,-0.761000,0.0,-1.0,0.0,-0.588431,0.0,0.153061,1.420196,-0.797342,-2.289240,-0.344748,-0.424753,-0.504103,2.510620,5.333333,-0.75,-1.250,-1.195727,-0.931776,-0.8,-1.000000,-0.75,-1.090909,-0.5,-1.060299,-0.944501
9615,0.895522,0.416667,-0.4,-0.172418,1.0,0.0,-1.0,-0.071427,5.0,-0.500000,-0.060454,0.540653,0.010833,-0.377617,0.783053,0.635908,-0.854557,3.166667,-0.75,-0.750,0.234564,0.355299,-0.7,-1.000000,-0.75,-0.909091,-0.5,-0.035363,-0.665431


In [166]:
features = columnas_df_vif

In [167]:
df_vif = df_vif[features].fillna(0)

In [168]:
df_vif_1 = pd.DataFrame()
df_vif_1['Feature'] = features
df_vif_1['VIF'] = [vif(df_vif[features], i) for i in range(len(features))]

In [169]:
df_vif_1.sort_values('VIF', ascending=False)

,Feature,VIF
20,avg_ss_duration_min,32.316476
22,total_used_transfer,27.939510
25,total_opened_push,18.060449
27,median_session_duration,15.247800
24,total_used_invest,13.408469
23,total_used_payment,12.141115
21,std_ss_duration_min,11.644107
11,txs_avg_amount,9.939396
13,total_amount_past30d,9.671985
12,avg_cashout_amount,8.962575


In [170]:
selected_vif = df_vif_1[df_vif_1['VIF'] < 10]
len(selected_vif)

22

In [171]:
selected_features = selected_vif['Feature'].values
selected_features = selected_features.tolist()

In [172]:
selected_features.append('Exited')

In [173]:
len(selected_features)

23

In [174]:
df_ml_filtered = df_ml[selected_features].copy()

# MODELADO

In [175]:
y = df_ml_filtered['Exited']
X = df_ml_filtered.drop('Exited', axis=1)

In [176]:
X = X.fillna(0)

In [177]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9617 entries, 0 to 9616
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 9617 non-null   int64  
 1   Age                         9617 non-null   int64  
 2   Tenure                      9617 non-null   int64  
 3   Balance                     9617 non-null   float64
 4   NumOfProducts               9617 non-null   int64  
 5   HasCrCard                   9617 non-null   int64  
 6   IsActiveMember              9617 non-null   int64  
 7   EstimatedSalary             9617 non-null   float64
 8   days_since_last_tx          9617 non-null   int64  
 9   total_tx_past90d            9617 non-null   int64  
 10  avg_week_txs                9617 non-null   float64
 11  txs_avg_amount              9617 non-null   float64
 12  avg_cashout_amount          9617 non-null   float64
 13  total_amount_past30d        9617 

In [178]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, stratify=y, random_state=42)

In [179]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7693 entries, 46 to 8497
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 7693 non-null   int64  
 1   Age                         7693 non-null   int64  
 2   Tenure                      7693 non-null   int64  
 3   Balance                     7693 non-null   float64
 4   NumOfProducts               7693 non-null   int64  
 5   HasCrCard                   7693 non-null   int64  
 6   IsActiveMember              7693 non-null   int64  
 7   EstimatedSalary             7693 non-null   float64
 8   days_since_last_tx          7693 non-null   int64  
 9   total_tx_past90d            7693 non-null   int64  
 10  avg_week_txs                7693 non-null   float64
 11  txs_avg_amount              7693 non-null   float64
 12  avg_cashout_amount          7693 non-null   float64
 13  total_amount_past30d        7693 non-

In [180]:
features = X_train.columns

In [181]:
features

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'days_since_last_tx',
       'total_tx_past90d', 'avg_week_txs', 'txs_avg_amount',
       'avg_cashout_amount', 'total_amount_past30d', 'amount_std',
       'cashout_ratio', 'transfer_ratio', 'days_since_last_ss',
       'total_ss_past30d', 'total_ss_past90d', 'total_failed_ss',
       'pct_sessions_with_transfer'],
      dtype='object')

In [182]:
len(features)

22

In [183]:
X_train.shape

(7693, 22)

In [184]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7693 entries, 46 to 8497
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 7693 non-null   int64  
 1   Age                         7693 non-null   int64  
 2   Tenure                      7693 non-null   int64  
 3   Balance                     7693 non-null   float64
 4   NumOfProducts               7693 non-null   int64  
 5   HasCrCard                   7693 non-null   int64  
 6   IsActiveMember              7693 non-null   int64  
 7   EstimatedSalary             7693 non-null   float64
 8   days_since_last_tx          7693 non-null   int64  
 9   total_tx_past90d            7693 non-null   int64  
 10  avg_week_txs                7693 non-null   float64
 11  txs_avg_amount              7693 non-null   float64
 12  avg_cashout_amount          7693 non-null   float64
 13  total_amount_past30d        7693 non-

In [185]:
features

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'days_since_last_tx',
       'total_tx_past90d', 'avg_week_txs', 'txs_avg_amount',
       'avg_cashout_amount', 'total_amount_past30d', 'amount_std',
       'cashout_ratio', 'transfer_ratio', 'days_since_last_ss',
       'total_ss_past30d', 'total_ss_past90d', 'total_failed_ss',
       'pct_sessions_with_transfer'],
      dtype='object')

In [79]:
len(features)

20

In [186]:
# Definir listas
binary_features = ['HasCrCard', 'IsActiveMember']
num_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
            'EstimatedSalary', 'days_since_last_tx',
           'total_tx_past90d', 'avg_week_txs', 'txs_avg_amount',
           'avg_cashout_amount', 'total_amount_past30d', 'amount_std',
           'cashout_ratio', 'transfer_ratio', 'days_since_last_ss',
           'total_ss_past30d', 'total_ss_past90d', 'total_failed_ss',
           'pct_sessions_with_transfer']

In [187]:
num_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'EstimatedSalary',
 'days_since_last_tx',
 'total_tx_past90d',
 'avg_week_txs',
 'txs_avg_amount',
 'avg_cashout_amount',
 'total_amount_past30d',
 'amount_std',
 'cashout_ratio',
 'transfer_ratio',
 'days_since_last_ss',
 'total_ss_past30d',
 'total_ss_past90d',
 'total_failed_ss',
 'pct_sessions_with_transfer']

In [188]:
len(num_cols)

20

In [189]:
# Escalar solo las numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

In [190]:
X_train_scaled

array([[-1.38099481e-01,  7.02358290e-03,  1.37598762e+00, ...,
        -1.53924185e+00, -1.18561419e+00, -1.42063150e+00],
       [-2.62170483e-01,  7.02358290e-03,  1.73706937e+00, ...,
        -2.79504879e-01,  9.23027579e-01, -2.50177608e-01],
       [ 2.05382155e+00,  1.96611033e-01,  1.73706937e+00, ...,
         2.09999828e+00, -6.58453750e-01,  2.17024235e+00],
       ...,
       [ 6.06326530e-01, -1.50967602e+00,  1.01490586e+00, ...,
        -1.39534105e-01, -6.58453750e-01, -5.25084565e-01],
       [ 1.16464604e+00,  7.02358290e-03,  6.53824107e-01, ...,
         1.40407444e-01,  3.95867136e-01,  1.81498059e-01],
       [ 1.06125354e+00, -3.72151317e-01, -4.29421159e-01, ...,
         4.36669515e-04, -1.31293307e-01, -9.63081321e-01]])

In [191]:
X_train[binary_features].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7693 entries, 46 to 8497
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   HasCrCard       7693 non-null   int64
 1   IsActiveMember  7693 non-null   int64
dtypes: int64(2)
memory usage: 180.3 KB


In [192]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_cols, index=X_train.index)
X_train_scaled[binary_features] = X_train[binary_features]
X_train_scaled

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,days_since_last_tx,total_tx_past90d,avg_week_txs,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,days_since_last_ss,total_ss_past30d,total_ss_past90d,total_failed_ss,pct_sessions_with_transfer,HasCrCard,IsActiveMember
46,-0.138099,0.007024,1.375988,0.983575,-0.921894,0.298517,-0.108546,-0.628839,1.072693,0.138601,1.909080,-0.074655,-1.281997,-1.281391,0.636109,0.864017,-0.964650,-1.539242,-1.185614,-1.420632,1,1
9567,-0.262170,0.007024,1.737069,0.855652,-0.921894,-0.070236,-0.108546,0.127001,0.016671,-0.205353,-0.089800,-0.185052,-0.006588,-0.294779,-0.233448,-0.532093,0.674203,-0.279505,0.923028,-0.250178,1,1
4872,2.053822,0.196611,1.737069,0.132389,-0.921894,1.699314,-0.108546,4.494077,2.445163,2.541865,2.634430,5.335157,1.648848,2.648465,-1.494918,-0.731538,2.968597,2.099998,-0.658454,2.170242,1,1
9064,0.358185,-0.087770,-0.429421,0.567020,0.796577,0.029706,-0.039394,-0.360096,-0.297341,-0.320644,-0.457438,-0.113595,-0.082596,-0.498833,-0.128488,-0.332649,-0.309109,-0.279505,0.923028,-0.336883,1,1
5510,0.389202,0.670580,1.014906,1.268860,-0.921894,-1.291833,-0.108546,0.252974,0.065976,-0.162344,-0.147409,-0.053196,-0.293125,0.216716,-0.331626,-0.532093,1.985285,0.420349,-0.131293,0.190649,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3557,0.182417,-0.656532,-1.151585,-1.220991,0.796577,-1.041705,0.237214,-0.645636,-0.377077,-0.961475,-1.568125,-0.526954,-0.668243,-0.758151,2.093557,1.329387,-1.292421,-0.699417,-0.658454,-1.420632,1,1
2713,0.451238,-0.182564,-1.512666,0.309061,0.796577,-0.002491,-0.073970,-0.788406,-0.565406,0.120577,1.176487,-0.513480,-0.381782,-1.574347,0.780248,-0.399131,-0.636880,-0.559446,-0.658454,-0.047981,1,0
8368,0.606327,-1.509676,1.014906,-0.048464,0.796577,-0.644810,-0.108546,0.563708,0.099145,-0.504441,-0.287284,0.193136,-0.955147,-0.240527,-0.183040,-0.532093,1.329744,-0.139534,-0.658454,-0.525085,1,0
3778,1.164646,0.007024,0.653824,-1.220991,0.796577,-1.410731,-0.108546,0.278169,0.152036,-0.623328,-0.495740,0.091073,-0.792714,-0.414928,0.228663,0.930498,-0.636880,0.140407,0.395867,0.181498,1,0


In [193]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7693 entries, 46 to 8497
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 7693 non-null   float64
 1   Age                         7693 non-null   float64
 2   Tenure                      7693 non-null   float64
 3   Balance                     7693 non-null   float64
 4   NumOfProducts               7693 non-null   float64
 5   EstimatedSalary             7693 non-null   float64
 6   days_since_last_tx          7693 non-null   float64
 7   total_tx_past90d            7693 non-null   float64
 8   avg_week_txs                7693 non-null   float64
 9   txs_avg_amount              7693 non-null   float64
 10  avg_cashout_amount          7693 non-null   float64
 11  total_amount_past30d        7693 non-null   float64
 12  amount_std                  7693 non-null   float64
 13  cashout_ratio               7693 non-

In [194]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns=num_cols, index=X_test.index)
X_test_scaled[binary_features] = X_test[binary_features]

In [195]:
X_train_scaled = X_train_scaled.fillna(0)
X_test_scaled = X_test_scaled.fillna(0)

In [196]:
X_test_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1924 entries, 246 to 4219
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 1924 non-null   float64
 1   Age                         1924 non-null   float64
 2   Tenure                      1924 non-null   float64
 3   Balance                     1924 non-null   float64
 4   NumOfProducts               1924 non-null   float64
 5   EstimatedSalary             1924 non-null   float64
 6   days_since_last_tx          1924 non-null   float64
 7   total_tx_past90d            1924 non-null   float64
 8   avg_week_txs                1924 non-null   float64
 9   txs_avg_amount              1924 non-null   float64
 10  avg_cashout_amount          1924 non-null   float64
 11  total_amount_past30d        1924 non-null   float64
 12  amount_std                  1924 non-null   float64
 13  cashout_ratio               1924 non

In [197]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7693 entries, 46 to 8497
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 7693 non-null   float64
 1   Age                         7693 non-null   float64
 2   Tenure                      7693 non-null   float64
 3   Balance                     7693 non-null   float64
 4   NumOfProducts               7693 non-null   float64
 5   EstimatedSalary             7693 non-null   float64
 6   days_since_last_tx          7693 non-null   float64
 7   total_tx_past90d            7693 non-null   float64
 8   avg_week_txs                7693 non-null   float64
 9   txs_avg_amount              7693 non-null   float64
 10  avg_cashout_amount          7693 non-null   float64
 11  total_amount_past30d        7693 non-null   float64
 12  amount_std                  7693 non-null   float64
 13  cashout_ratio               7693 non-

In [198]:
X_test_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1924 entries, 246 to 4219
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CreditScore                 1924 non-null   float64
 1   Age                         1924 non-null   float64
 2   Tenure                      1924 non-null   float64
 3   Balance                     1924 non-null   float64
 4   NumOfProducts               1924 non-null   float64
 5   EstimatedSalary             1924 non-null   float64
 6   days_since_last_tx          1924 non-null   float64
 7   total_tx_past90d            1924 non-null   float64
 8   avg_week_txs                1924 non-null   float64
 9   txs_avg_amount              1924 non-null   float64
 10  avg_cashout_amount          1924 non-null   float64
 11  total_amount_past30d        1924 non-null   float64
 12  amount_std                  1924 non-null   float64
 13  cashout_ratio               1924 non

## BALANCE

In [199]:
smote = SMOTE(random_state=42)
X_os, y_os = smote.fit_resample(X_train_scaled, y_train)

In [200]:
X_os

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,days_since_last_tx,total_tx_past90d,avg_week_txs,txs_avg_amount,avg_cashout_amount,total_amount_past30d,amount_std,cashout_ratio,transfer_ratio,days_since_last_ss,total_ss_past30d,total_ss_past90d,total_failed_ss,pct_sessions_with_transfer,HasCrCard,IsActiveMember
0,-0.138099,0.007024,1.375988,0.983575,-0.921894,0.298517,-0.108546,-0.628839,1.072693,0.138601,1.909080,-0.074655,-1.281997,-1.281391,0.636109,0.864017,-0.964650,-1.539242,-1.185614,-1.420632,1,1
1,-0.262170,0.007024,1.737069,0.855652,-0.921894,-0.070236,-0.108546,0.127001,0.016671,-0.205353,-0.089800,-0.185052,-0.006588,-0.294779,-0.233448,-0.532093,0.674203,-0.279505,0.923028,-0.250178,1,1
2,2.053822,0.196611,1.737069,0.132389,-0.921894,1.699314,-0.108546,4.494077,2.445163,2.541865,2.634430,5.335157,1.648848,2.648465,-1.494918,-0.731538,2.968597,2.099998,-0.658454,2.170242,1,1
3,0.358185,-0.087770,-0.429421,0.567020,0.796577,0.029706,-0.039394,-0.360096,-0.297341,-0.320644,-0.457438,-0.113595,-0.082596,-0.498833,-0.128488,-0.332649,-0.309109,-0.279505,0.923028,-0.336883,1,1
4,0.389202,0.670580,1.014906,1.268860,-0.921894,-1.291833,-0.108546,0.252974,0.065976,-0.162344,-0.147409,-0.053196,-0.293125,0.216716,-0.331626,-0.532093,1.985285,0.420349,-0.131293,0.190649,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12273,-0.213963,0.702248,-1.151585,0.719979,-0.921894,1.025421,-0.004933,-0.816387,-0.506731,0.385299,0.190626,-0.407830,1.218289,-0.655476,0.137292,2.216077,-1.292421,-1.492637,-0.658454,0.714143,0,1
12274,0.621171,0.533876,-0.493277,0.788824,-0.921894,-0.397123,-0.013990,-0.711849,-0.322897,-0.523600,0.265131,-0.475315,-1.129777,0.298465,-1.221031,-0.066723,-0.665862,-0.864141,-0.705067,-0.157054,1,0
12275,0.928605,0.827404,-0.304624,-1.220991,-0.921894,0.283595,-0.063294,-0.537388,-0.287833,-0.152674,-0.050877,-0.275652,0.041080,-0.490791,-0.165231,3.021752,-1.292421,-1.399271,-0.294059,-0.641954,1,0
12276,0.391382,0.778216,-0.839422,-0.175169,-0.921894,0.192903,0.085823,-0.743001,-0.400850,-0.574264,0.041051,-0.485031,-0.960661,-0.509770,-1.156123,3.526989,-1.292421,-1.225285,-0.131293,-0.595646,1,0


In [201]:
rf = RandomForestClassifier(random_state=42, max_depth=10)

In [202]:
rf.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [203]:
def metricas_evaluacion(modelo, 
                       x_test: pd.core.frame.DataFrame,
                       y_test: pd.core.series.Series,
                       nombre: str,
                       conjunto: str,
                       umbral: float=None):
    """
    Funcion que calcula métricas de modelos clasificación
    ------------------------------------------------------------
    Parámetros:
    
        modelo:
            Modelo de Machine Learning Scikit-learn (debe contar con el método predict)
            
        x_test: pandas.core.frame.DataFrame
            DataFrame con las variables explicativas utilizadas para evaluar el modelo
            
        y_test:
            Pandas Series con las etiquetas reales de los datos utilizados para evaluar el modelo
            
        nombre: str
            Nombre que identificará el modelo para comparaciones
            
        conjunto: str
            Nombre del conjunto de datos utilizados para entrenar y evaluar el modelo
    ------------------------------------------------------------
    Returns:
        Diccionario con nombre del modelo, dataset utilizado y metricas: Accuracy, Precision, Recall, F1-score
    """
    
    if umbral == None:
        metricas = {}
        y_pred = modelo.predict(x_test)
        metricas['Model'] = nombre
        metricas['Dataset'] = conjunto
        metricas['Accuracy'] = np.round(accuracy_score(y_test, y_pred), 4)
        metricas['Precision'] = np.round(precision_score(y_test, y_pred), 4)
        metricas['Recall'] = np.round(recall_score(y_test, y_pred), 4)
        metricas['F1-score'] = np.round(f1_score(y_test, y_pred), 4)
        if hasattr(modelo, "predict_proba"):
            y_proba = modelo.predict_proba(x_test)[:, 1]
            metricas['AUC'] = np.round(roc_auc_score(y_test, y_proba), 4)
        else:
            metricas['AUC'] = '---'
        metricas['Umbral'] = 0.50
    else:
        if hasattr(modelo, "predict_proba"):
            metricas = {}
            probs = modelo.predict_proba(x_test)[:, 1]
            y_pred = (probs >= umbral).astype(int)
            metricas['Model'] = nombre
            metricas['Dataset'] = conjunto
            metricas['Accuracy'] = np.round(accuracy_score(y_test, y_pred), 4)
            metricas['Precision'] = np.round(precision_score(y_test, y_pred), 4)
            metricas['Recall'] = np.round(recall_score(y_test, y_pred), 4)
            metricas['F1-score'] = np.round(f1_score(y_test, y_pred), 4)
            metricas['AUC'] = np.round(roc_auc_score(y_test, probs), 4)
            metricas['Umbral'] = umbral
        else:
            raise ValueEror('El modelo no cuenta con el método "predict_proba"')
    
    return metricas

In [204]:
rf_results = metricas_evaluacion(modelo=rf,
                           x_test=X_test_scaled,
                           y_test=y_test,
                           nombre='RandomForest',
                           conjunto='X_sacled')

In [205]:
rf_results

{'Model': 'RandomForest',
 'Dataset': 'X_sacled',
 'Accuracy': 0.9413,
 'Precision': 0.9452,
 'Recall': 0.7532,
 'F1-score': 0.8383,
 'AUC': 0.9829,
 'Umbral': 0.5}

In [206]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = df_ml_filtered.corr()['Exited'].sort_values(ascending=False)

print(corr)

Exited                        1.000000
days_since_last_ss            0.405820
Age                           0.287095
pct_sessions_with_transfer    0.169046
cashout_ratio                 0.124505
avg_cashout_amount            0.121349
Balance                       0.118958
txs_avg_amount                0.115830
avg_week_txs                  0.090928
days_since_last_tx            0.071204
amount_std                    0.053987
EstimatedSalary               0.015917
HasCrCard                    -0.009256
Tenure                       -0.013700
CreditScore                  -0.026340
total_amount_past30d         -0.029040
NumOfProducts                -0.045492
transfer_ratio               -0.098862
total_tx_past90d             -0.119975
total_failed_ss              -0.122583
IsActiveMember               -0.157051
total_ss_past90d             -0.267338
total_ss_past30d             -0.314354
Name: Exited, dtype: float64


In [105]:
rf.feature_importances_

array([0.0027369 , 0.0249431 , 0.00847577, 0.00485568, 0.01475637,
       0.00066204, 0.00263225, 0.00355699, 0.02907787, 0.01142285,
       0.0194185 , 0.03562911, 0.00663349, 0.00764212, 0.01103363,
       0.33578209, 0.26991344, 0.197852  , 0.00200159, 0.01097419])

In [106]:
pd.DataFrame({'Feature': X_train_scaled.columns, 'Importancia': rf.feature_importances_}).sort_values('Importancia', ascending=False)

,Feature,Importancia
15,days_since_last_ss,0.335782
16,total_ss_past30d,0.269913
17,total_ss_past90d,0.197852
11,total_amount_past30d,0.035629
8,days_since_last_tx,0.029078
1,Age,0.024943
10,avg_cashout_amount,0.019418
4,NumOfProducts,0.014756
9,txs_avg_amount,0.011423
14,transfer_ratio,0.011034
